**59.¿Cuál es el idioma para el cual sus usuarios realizan más agradecimientos en promedio? ¿Y el de menos agradecimientos? Calcule lo mismo para quienes reciben agradecimientos. (⭐⭐⭐)**

In [ ]:
import pandas as pd 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Leo solo las columnas que voy a usar, ya sabiendo como es el .csv

In [ ]:
df_logs = pd.read_csv( "/content/drive/MyDrive/datosTP1/logs.csv", usecols = ['contributor_username', 'contributor_id', 'contributor_ip', 'action', 'title'] )

<ipython-input-2-320f4fa2fdb9>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_logs = pd.read_csv( "/content/drive/MyDrive/datosTP1/logs.csv", usecols = ['contributor_username', 'contributor_id', 'contributor_ip', 'action', 'title'] )


Veo los tipos de "action" que hay, encontrando el "thank" que busco.


In [ ]:
df_logs["action"].unique()

array(['delete', 'block', 'upload', 'protect', 'restore', 'unblock',
       'unprotect', 'rights', 'move', 'move_redir', nan, 'renameuser',
       'newusers', 'create', 'create2', 'modify', 'patrol', 'autocreate',
       'usergroups', 'move_prot', 'reblock', 'event', 'revision',
       'interwiki', 'byemail', 'thank', 'add', 'update', 'remove',
       'selfadd', 'selfremove', 'review', 'failure', 'skipbadns', 'send',
       'unreview', 'skipnouser', 'profilesave', 'undelete', 'merge',
       'mergeuser', 'deleteuser', 'delete_redir', 'whitelist', 'change',
       'new', 'addlink'], dtype=object)

Creo un nuevo df solo con aquellas filas que cumplan la condición de que sea de agradecimiento.


In [ ]:
df_agradecimientos = df_logs.loc[ df_logs["action"] == "thank" ]

Veo que para los agradecimientos contributor_ip es siempre NaN.


In [ ]:
df_agradecimientos["contributor_ip"].unique()

array([nan], dtype=object)

Droppeo la columna de contributor_ip que son NaN's, para liberar memoria y que sea más legible, así trabajar más cómodo.


In [ ]:
df_agradecimientos = df_agradecimientos.drop( ["contributor_ip"], axis=1 )

Libero memoria quedándome solo con la versión reducida del df_logs original.


In [ ]:
del df_logs

In [ ]:
df_agradecimientos.head()

,contributor_username,contributor_id,action,title
5089429,Anonyme Wikipe-tan,2792293.0,thank,Usuario:Isha
5089430,Anonyme Wikipe-tan,2792293.0,thank,Usuario:Osado
5089443,Fabrice Florin (WMF),2756867.0,thank,Usuario:Fabrice Florin
5089554,David Steven,2239930.0,thank,Usuario:Sala Zero
5089560,Renly,1837330.0,thank,Usuario:StockZXD


Chequeo que no haya alguna fila con el contributor_id y el title como NaN, porque no me serviría.

In [ ]:
df_agradecimientos.loc[ df_agradecimientos.contributor_id.isna() & df_agradecimientos.title.isna() ] 

,contributor_username,contributor_id,action,title


Busco el id de los usuarios que fueron agradecidos en contents.csv, sabiendo que necesito su id para conocer el idioma que hablan. Y sabiendo que fueron agradecidos por una revisión que hicieron.

In [ ]:
df_contents = pd.read_csv("/content/drive/MyDrive/datosTP1/contents.csv", usecols=["revisor_username", "revisor_id"])

In [ ]:
df_contents

,revisor_username,revisor_id
0,Metrónomo,1708233.0
1,NaN,NaN
2,Traitrich,4642283.0
3,Lojwe,5338508.0
4,Jialxv,3615366.0
...,...,...
4132159,Copydays,474907.0
4132160,Crls9011,4032712.0
4132161,Gabriel Solo,158577.0
4132162,Términus,5429951.0


Me quedo solo con los usernames que no estén repetidos, para hacer un merge más liviano.


In [ ]:
df_contents = df_contents.loc[ df_contents.revisor_username.notna() ]
df_revisores = df_contents.drop_duplicates( subset="revisor_username" )

In [ ]:
del df_contents

In [ ]:
df_revisores

,revisor_username,revisor_id
0,Metrónomo,1708233.0
2,Traitrich,4642283.0
3,Lojwe,5338508.0
4,Jialxv,3615366.0
5,PatriHorrillo,689866.0
...,...,...
4131723,Templajaen,6332400.0
4131771,Islasazules,6291332.0
4131957,BlockchainWolf,6331508.0
4132053,Pommelito crazy,6252228.0


In [ ]:
df_agradecimientos.title.isna().any()

True

Busco dejar el title de df_agradecimientos con el mismo formato que el revisor_username de df_revisores para poder mergearlos. Si es NaN lo dejo igual.

In [ ]:
df_agradecimientos.title = df_agradecimientos.title.map( lambda x: x.split(":")[1] if isinstance(x, str) else x )

Renombro para mergear por mismo valor en las columnas revisor_username del df_revisores y la columna title de df_agradecimientos, que representan lo mismo, sabiendo que el revisor es quien fue agradecido.

In [ ]:
df_revisores = df_revisores.rename( columns = {"revisor_username": "agradecido_username"} )
df_agradecimientos = df_agradecimientos.rename( columns = {"title": "agradecido_username"} )

Mergeo y me quedo con todos los elementos de df_agradecimientos, pero ahora también me quedo con los elementos del df_contents cuyos agradecido_username coincidan con el de df_agradecimientos. Así poder conocer el id de los que estén en df_agradecimientos.

In [ ]:
df_agradecimientos = df_agradecimientos.merge(df_revisores, on="agradecido_username", how="left")

Renombro las columnas sabiendo que el contributor es el agradecedor y el revisor el agradecido, para que sea más legible.

In [ ]:
df_agradecimientos = df_agradecimientos.rename( columns = {"contributor_username": "agradecedor_username", "contributor_id": "agradecedor_id", "revisor_id": "agradecido_id"} )

In [ ]:
df_agradecimientos

,agradecedor_username,agradecedor_id,action,agradecido_username,agradecido_id
0,Anonyme Wikipe-tan,2792293.0,thank,Isha,177087.0
1,Anonyme Wikipe-tan,2792293.0,thank,Osado,612673.0
2,Fabrice Florin (WMF),2756867.0,thank,Fabrice Florin,NaN
3,David Steven,2239930.0,thank,Sala Zero,1888854.0
4,Renly,1837330.0,thank,StockZXD,2702856.0
...,...,...,...,...,...
241943,Jorge Calbera,6325764.0,thank,Tector,1237851.0
241944,JulioW4rrior00,5950160.0,thank,Latothy99,6001164.0
241945,Latothy99,6001164.0,thank,JulioW4rrior00,5950160.0
241946,BGtanl,5527055.0,thank,Alpinu,750788.0


In [ ]:
df_languages = pd.read_csv("/content/drive/MyDrive/datosTP1/languages.csv")

Veo como es el dataframe. Veo que contiene los idiomas que hablan algunos usuarios registrados.

In [ ]:
df_languages.head()

,babel_user,babel_lang,babel_level
0,4502458,FR,2
1,5928200,aa,0
2,46918,ab,0
3,2050449,ab,1
4,4715583,ace,0


In [ ]:
df_agradecimientos.agradecedor_id.nunique()

26369

In [ ]:
df_languages.babel_user.nunique()

10815

Veo que conozco que idiomas hablan solo menos de la mitad de los usuarios que agradecieron.

In [ ]:
df_languages.babel_level.isna().any()

False

In [ ]:
df_languages.babel_user.isna().any()

False

In [ ]:
df_languages.babel_lang.isna().any()

True

El anexo de Nivel en el lenguaje dice que si el "babel_level" == 0, entonces no se sabe hablar ese idioma. Me quedo solo con los idiomas que cada usuario puede hablar y que no sean NaN.


In [ ]:
df_languages = df_languages.loc[ df_languages.babel_lang.notna() ]
df_lenguajes_hablados = df_languages.loc[ df_languages.babel_level != "0" ]

In [ ]:
del df_languages

Ví que hay códigos de idioma en mayúsculas, los convierto todos a minúsculas para que se agrupen correctamente luego.


In [ ]:
df_lenguajes_hablados["babel_lang"] = df_lenguajes_hablados["babel_lang"].str.lower()

In [ ]:
df_cantidades_agradecedores = df_agradecimientos.groupby( ["agradecedor_id"] ).size().reset_index( name="agradecimientos_realizados" )

In [ ]:
df_cantidades_agradecedores

,agradecedor_id,agradecimientos_realizados
0,415.0,1
1,719.0,2
2,931.0,1
3,967.0,1
4,1005.0,1
...,...,...
26364,6333821.0,1
26365,6333922.0,1
26366,6333959.0,1
26367,6333984.0,1


In [ ]:
df_cantidades_agradecidos = df_agradecimientos.groupby( ["agradecido_id"] ).size().reset_index( name="agradecimientos_recibidos" )

In [ ]:
df_cantidades_agradecidos

,agradecido_id,agradecimientos_recibidos
0,1.0,12
1,7.0,1
2,30.0,1
3,43.0,3
4,311.0,1
...,...,...
8924,6326049.0,1
8925,6326514.0,2
8926,6328583.0,1
8927,6331808.0,1


Renombro para mergear.


In [ ]:
df_cantidades_agradecedores = df_cantidades_agradecedores.rename( columns = {"agradecedor_id": "id"} )
df_cantidades_agradecidos = df_cantidades_agradecidos.rename( columns = {"agradecido_id": "id"} )
df_lenguajes_hablados = df_lenguajes_hablados.rename( columns = {"babel_user": "id"} )

Mergeo quedándome con los ids de todos los usuarios que sé que idioma hablan y donde coincida el id me quedo también con cuantos agradecimientos realizó...

Para el lenguaje menos agradecedor lo calculo tanto para el que haya promedio 0, como para el que al menos un usuario haya agradecido.

In [ ]:
df_cantidades_agradecedores = df_lenguajes_hablados.merge(df_cantidades_agradecedores, on="id", how="left")
df_cantidades_agradecedores.agradecimientos_realizados = df_cantidades_agradecedores.agradecimientos_realizados.fillna(0)
df_cantidades_agradecedores = df_cantidades_agradecedores.groupby(["babel_lang"]).agg( { "agradecimientos_realizados": ["mean"] } )
df_cantidades_agradecedores.columns = ["promedio"]

max_promedio_realizados = df_cantidades_agradecedores.promedio.max()
min_promedio_realizados = df_cantidades_agradecedores.loc[ df_cantidades_agradecedores.promedio > 0 ].promedio.min()


df_cantidades_agradecedores.loc[ (df_cantidades_agradecedores.promedio >= max_promedio_realizados) | (df_cantidades_agradecedores.promedio == min_promedio_realizados) ]

,promedio
babel_lang,
nn,0.090909
tk,114.000000


Los usuarios que hablan turcomano (de Turkmenistán) agradecieron más en promedio. Mientras que los usuarios que hablan noruego agradecieron menos en promedio (de los que agradecieron al menos una vez).

In [ ]:
min_real_promedio_realizados = df_cantidades_agradecedores.promedio.min()

df_cantidades_agradecedores.loc[ df_cantidades_agradecedores.promedio <= min_real_promedio_realizados ].index

Index(['ab', 'acf', 'aln', 'am', 'arc', 'ary', 'avk', 'azb', 'bcl', 'bew',
       'bjn', 'br', 'cak', 'cas', 'cbk', 'ch', 'chr', 'ckb', 'cmn', 'cnt',
       'csb', 'csn', 'cu', 'cv', 'cy', 'eml', 'fil', 'fur', 'fy', 'gan', 'gd',
       'gom-latn', 'gsw', 'gym', 'haw', 'hsb', 'ilo', 'jbo', 'kbp', 'kea',
       'kl', 'km', 'krl', 'ku', 'kw', 'lij', 'ln', 'ltg', 'lzh', 'lzz',
       'map-bms', 'mfs', 'mi', 'min', 'ml', 'mnw', 'mr', 'mt', 'mur', 'my',
       'nap', 'nb', 'nla', 'nov', 'ots', 'pag', 'pam', 'pcd', 'pfl', 'pnb',
       'ppl', 'qub', 'rgn', 'rue', 'rup', 'rw', 'sdh', 'se', 'shn', 'si', 'su',
       'ta', 'te', 'tg', 'ti', 'tmr', 'tzm', 'ur', 'wa', 'war', 'wuu', 'yue',
       'zea', 'zh-hans', 'zh-hans-cn', 'zh-hant', 'zh-hant-hk', 'zh-hant-tw',
       'zza'],
      dtype='object', name='babel_lang')

Veo que los que menos agradecieron **realmente** en promedio son los usuarios que hablan todos los idiomas de arriba, con promedio de 0.

In [ ]:
min_real_promedio_realizados

0.0

Mergeo quedándome con los ids de todos los usuarios que sé que idioma hablan y donde coincida el id me quedo también con cuantos agradecimientos recibió...

Para el lenguaje menos agradecido lo calculo tanto para el que haya promedio 0, como para el que al menos un usuario haya sido agradecido.


In [ ]:
df_cantidades_agradecidos = df_lenguajes_hablados.merge(df_cantidades_agradecidos, on="id", how="left")
df_cantidades_agradecidos.agradecimientos_recibidos = df_cantidades_agradecidos.agradecimientos_recibidos.fillna(0)
df_cantidades_agradecidos = df_cantidades_agradecidos.groupby(["babel_lang"]).agg( { "agradecimientos_recibidos": ["mean"] } )
df_cantidades_agradecidos.columns = ["promedio"]

max_promedio_recibidos = df_cantidades_agradecidos.promedio.max()
min_promedio_recibidos = df_cantidades_agradecidos.loc[ df_cantidades_agradecidos.promedio > 0 ].promedio.min()

df_cantidades_agradecidos.loc[ (df_cantidades_agradecidos.promedio >= max_promedio_recibidos) | (df_cantidades_agradecidos.promedio == min_promedio_recibidos) ]

,promedio
babel_lang,
nds,0.1
tk,182.0


Los usuarios que hablan turcomano recibieron más agradecimientos en promedio. 
Y los usuarios que hablan bajo sajón o bajo alemán occidental son los que menos agradecimientos recibieron en promedio (de los que fueron agradecidos al menos una vez).

In [ ]:
min_real_promedio_recibidos = df_cantidades_agradecidos.promedio.min()

for i in (df_cantidades_agradecidos.loc[ df_cantidades_agradecidos.promedio <= min_real_promedio_recibidos ].index):
  print(i)

ab
aln
am
ary
avk
azb
bar
bcl
bew
bjn
bn
br
cas
cbk
ch
chr
ckb
cmn
csc
csn
cu
cv
de-at
en-ca
fil
fo
fur
fy
gd
gom-latn
gym
haw
ilo
ise
jbo
kbp
kea
kk
kl
ku
kw
ky
lb
lij
ln
lo
ltg
lv
lzh
lzz
mad
map-bms
mfs
mi
min
ml
mnw
mur
my
nia
nla
non
nov
oj
ots
pag
pam
pcd
pfl
pnb
ppl
qub
rgn
rup
rw
sdh
shn
si
sm
sq
su
ta
te
tg
ti
tpi
tzh
tzm
tzo
ur
wa
war
wuu
yua
yue
zap
zea
zh-hans
zh-hans-cn
zh-hant
zh-hant-hk
zh-hant-tw
zza


In [ ]:
min_real_promedio_recibidos

0.0

Todos los idiomas de arriba son para los que sus usuarios recibieron menos agradecimientos **realmente** en promedio, con promedio 0.